In [14]:
import pandas as pd
import matplotlib.pyplot as mp
from sklearn.model_selection import train_test_split as TTS
from sklearn import set_config
from sklearn.metrics import accuracy_score as AS
import seaborn as sn
import plotly.express as px
import numpy as np
from nltk.stem import WordNetLemmatizer as WNL
from nltk import word_tokenize
import nltk
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [15]:
import pandas as pd
import numpy as np
from keras.datasets import mnist
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,BatchNormalization,SimpleRNN,Embedding,Dropout,LSTM
from tensorflow.keras.callbacks import EarlyStopping as ES
import matplotlib.pyplot as mp
from sklearn.preprocessing import StandardScaler as SS
from keras.utils import pad_sequences
from keras.models import Model
import string
import nltk

In [18]:
t=pd.read_csv("imdb_labelled.txt",delimiter="\t",header=None,names=["review","sentiment"])
t

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
804,I just got bored watching Jessice Lange take h...,0
805,"Unfortunately, any virtue in this film's produ...",0
806,"In a word, it is embarrassing.",0
807,Exceptionally bad!,0


In [19]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [20]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag[0]=='J':
        return wordnet.ADJ
    elif treebank_tag[0]=='V':
        return wordnet.VERB
    elif treebank_tag[0]=='R':
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [21]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WNL()
    def __call__(self, articles):
        lem=list()
        return [self.wnl.lemmatize(t.lower(),pos=get_wordnet_pos(nltk.pos_tag(list(t.lower()))[0][1])) for t in word_tokenize(articles) if t not in set(stopwords.words('english')) and (t not in string.punctuation+"``"+"..."+"--") and (t.isnumeric()==False)]

In [22]:
x=t["review"]
x

0      A very, very, very slow-moving, aimless movie ...
1      Not sure who was more lost - the flat characte...
2      Attempting artiness with black & white and cle...
3           Very little music or anything to speak of.  
4      The best scene in the movie was when Gerardo i...
                             ...                        
804    I just got bored watching Jessice Lange take h...
805    Unfortunately, any virtue in this film's produ...
806                     In a word, it is embarrassing.  
807                                 Exceptionally bad!  
808    All in all its an insult to one's intelligence...
Name: review, Length: 809, dtype: object

In [23]:
y=np.where(t.sentiment=="positive",1,0)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [24]:
tok=tf.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters=None,
    lower=True,
    split=' ',
    char_level=False,
    oov_token="<nothing>",
    analyzer=LemmaTokenizer(),
)

In [25]:
tok.fit_on_texts(x)


In [26]:
tok.word_index


{'<nothing>': 1,
 'i': 2,
 'movie': 3,
 'the': 4,
 'film': 5,
 "'s": 6,
 "n't": 7,
 'it': 8,
 'one': 9,
 'this': 10,
 'bad': 11,
 'character': 12,
 'make': 13,
 'good': 14,
 'like': 15,
 'watch': 16,
 'time': 17,
 'acting': 18,
 'really': 19,
 'great': 20,
 'even': 21,
 "''": 22,
 'see': 23,
 'story': 24,
 'well': 25,
 'ever': 26,
 'actor': 27,
 'but': 28,
 'plot': 29,
 'would': 30,
 'could': 31,
 'best': 32,
 'cast': 33,
 'also': 34,
 'thing': 35,
 'script': 36,
 'a': 37,
 'work': 38,
 'love': 39,
 'seen': 40,
 'way': 41,
 'waste': 42,
 'look': 43,
 'give': 44,
 'think': 45,
 'there': 46,
 'real': 47,
 'every': 48,
 'funny': 49,
 'much': 50,
 'get': 51,
 'line': 52,
 'if': 53,
 'say': 54,
 'scenes': 55,
 'show': 56,
 'better': 57,
 'performance': 58,
 'year': 59,
 'never': 60,
 'wonderful': 61,
 'little': 62,
 'go': 63,
 'excellent': 64,
 'and': 65,
 'everything': 66,
 'anyone': 67,
 'part': 68,
 'totally': 69,
 'know': 70,
 'music': 71,
 'scene': 72,
 'people': 73,
 'life': 74,
 'not

In [37]:
y.shape


(809,)

In [39]:
x.shape



(809,)

In [40]:
xta,xte,yta,yte=TTS(x,y,random_state=2,test_size=0.2)


In [45]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])


In [46]:
es=ES(monitor="val_loss",
    min_delta=0.00000001,
    patience=10,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0,)